In [1]:
import logging, sys, os, re
sys.path.append('/home/sogilvy/repos/timbermafia/tests')
from columntests import TMString

In [2]:
def transform_record(record):
    new_d = {}
    for key, value in record.__dict__.items():
        if isinstance(value, str):
            new_d[key] = TMString(value)
    record.__dict__.update(new_d)

class MyFormatter(logging.Formatter):
#     def __init__(self, *args, **kwargs):
#         pass
    
    def format(self, record):
        # Convert the message string to an TMString with enhanced fmt_spec
        record.message = TMString(record.getMessage())
        
        # Convert the formatted ascitime string to an TMString with enhanced fmt_spec
        if self.usesTime():
            record.asctime = TMString(self.formatTime(record, self.datefmt))
        
        # Convert any other miscellanous records
        transform_record(record)
        
        s = self.formatMessage(record)
        if record.exc_info:
            if not record.exc_text:
                record.exc_text = self.formatException(record.exc_info)
        if record.exc_text:
            if s[-1:] != "\n":
                s = s + "\n"
            s = s + record.exc_text
        if record.stack_info:
            if s[-1:] != "\n":
                s = s + "\n"
            s = s + self.formatStack(record.stack_info)
        return s

In [3]:
from timbermafia.rainbow import RainbowStreamHandler
h = RainbowStreamHandler()
f = MyFormatter('{asctime:u} _| {levelname} _| {name:>50}.{funcName:>69} __>> {message:b}', '%H:%M:%S', style='{', validate=False)
h.setFormatter(f)
l = logging.getLogger(__name__)
l.addHandler(h)
l.setLevel(logging.DEBUG)

In [4]:
def test_log():
    l.info('I have some big, big news for you, hence this text is bold')
    l.debug('some debug info for you there chief')
test_log()

20
['\x1b\x1b12:57:24\x1b', '_|', 'INFO\x1b', '_|', '\x1b__main__\x1b.\x1btest_log\x1b', '__>>', '\x1bI', 'have', 'some', 'big,', 'big', 'news', 'for', 'you,', 'hence', 'this', 'text', 'is', 'bold\x1b\x1b']
12:57:24 _| INFO _| __main__.test_log __>> I have some big, big news for you, hence this text is bold
10
['\x1b\x1b12:57:24\x1b', '_|', 'DEBUG\x1b', '_|', '\x1b__main__\x1b.\x1btest_log\x1b', '__>>', '\x1bsome', 'debug', 'info', 'for', 'you', 'there', 'chief\x1b\x1b']
12:57:24 _| DEBUG _| __main__.test_log __>> some debug info for you there chief


In [5]:
test = '>> _| {asctime:u} _| {levelname} _| {name:>50}.{funcName:>69} __>> {message:b}'
pattern = '_{1,2}\W{1,2}'

re.split(pattern, test)
re.findall(pattern, test)

['>> ',
 '{asctime:u} ',
 '{levelname} ',
 '{name:>50}.{funcName:>69} ',
 ' {message:b}']

['_| ', '_| ', '_| ', '__>>']

In [6]:
p = f'{pattern}.*?{pattern}|^.*?{pattern}|{pattern}.*?$'
p

'_{1,2}\\W{1,2}.*?_{1,2}\\W{1,2}|^.*?_{1,2}\\W{1,2}|_{1,2}\\W{1,2}.*?$'

In [7]:
re.findall(p, test)
# i = re.finditer(p, test)
# m = i.__next__()

['>> _| ', '_| {levelname} _| ', '__>> {message:b}']

In [8]:
sep = '_{1,2}\S+|_{1,2}' # separator pattern
text = 's _| {asctime:u} _| level: {levelname:>80} _ {name}.{funcName} __>> {message}'

# Negative lookbehind needs fixed length, but separators vary.
# Take advantage of the fact that whatever escapes the
# separator tags is explicitly not in the rest of the format string.
# In this case, '_'
partial_text = re.sub(sep, '_', text)
partial_text

csp = '.*\{\w+(:\S+)?\}.*' # column sub pattern {name} {message:>71,b}

parts = partial_text.split('_')
column_dict = {k: {'contents': v} for k, v in enumerate(parts) if re.match(csp, v)}

column_dict

ntext = text
for i, d in column_dict.items():
    s = d['contents']
    ntext = ntext.replace(s, str(i))
ntext

csp2 = '\{\S+?\}'

for k, d in column_dict.items():
    s = d['contents']
    d['fields'] = re.findall('(?<=\{)[a-zA-Z]+(?=[\}:])', s)
    
column_dict

's _ {asctime:u} _ level: {levelname:>80} _ {name}.{funcName} _ {message}'

{1: {'contents': ' {asctime:u} '},
 2: {'contents': ' level: {levelname:>80} '},
 3: {'contents': ' {name}.{funcName} '},
 4: {'contents': ' {message}'}}

's _|1_|2_3__>>4'

{1: {'contents': ' {asctime:u} ', 'fields': ['asctime']},
 2: {'contents': ' level: {levelname:>80} ', 'fields': ['levelname']},
 3: {'contents': ' {name}.{funcName} ', 'fields': ['name', 'funcName']},
 4: {'contents': ' {message}', 'fields': ['message']}}